In [ ]:
import vrplib
import pathlib


from itertools import pairwise
import plotly.express as px
import plotly.offline as py
# import chart_studio.plotly as py
import pandas as pd

py.init_notebook_mode()

In [ ]:
# one can try out other problems, but take note that
# A series problems seem not to include the customer location data
# (it is used to visualize the results, but not in the actual ALNS algorithm, actually)
# instance_name = "E-n13-k4"

# instance_name = "E-n76-k10"
# instance_name = "B-n63-k10"
# instance_name = "E-n23-k3"
# instance_name = "E-n101-k14"
instance_name = "E-n101-k8"

instance_path = pathlib.Path("../data/") / f"{instance_name}.vrp"
solution_path = pathlib.Path("../data/") / f"{instance_name}.sol"

if not instance_path.is_file():
    vrplib.download_instance(instance_name, instance_path)
if not solution_path.is_file():
    vrplib.download_solution(instance_name, solution_path)

data = vrplib.read_instance(instance_path)
best_known_solution = vrplib.read_solution(solution_path)

In [ ]:
data.keys()

In [ ]:
print(data["name"])
print(data["comment"])
print(data["dimension"])

In [ ]:
data["edge_weight"].shape

In [ ]:
# best_known_solution

In [ ]:
import lns
from lns import cvrp

In [ ]:
# data

In [ ]:
problem = cvrp.Problem.from_vrplib(data)
opt_sol = cvrp.Solution.from_vrplib(best_known_solution)

In [ ]:
def plot_solution(p: cvrp.Problem, sol: cvrp.Solution):
    df = pd.DataFrame(
        {
            "x": p.customers[:, 0],
            "y": p.customers[:, 1],
            "customer": range(len(p.customers)),
            "marker": "customer",
            "demand": p.demands,
            "marker_size": 0.2,
        }
    )

    df.loc[0, ["marker", "marker_size"]] = "depot", 3

    fig = px.scatter(
        df,
        x="x",
        y="y",
        symbol="marker",
        symbol_sequence=["star", "circle-open"],
        size="marker_size",
        # color="demand",
        # color_continuous_scale="inferno",
        hover_data="customer",
        # template="ploly_white",
        title=p.name,
        height=800,
        width=1200,
    )

    for i, route in enumerate(sol.routes, start=1):
        edge_x, edge_y = [], []

        for a, b in pairwise([0] + route + [0]):
            edge_x.append(p.customers[a, 0])
            edge_x.append(p.customers[b, 0])
            edge_x.append(None)

            edge_y.append(p.customers[a, 1])
            edge_y.append(p.customers[b, 1])
            edge_y.append(None)

        fig.add_scatter(
            x=edge_x,
            y=edge_y,
            name=f"Route {i}",
            showlegend=False,
        )

    return fig

In [ ]:
import numpy as np


rng = np.random.default_rng(seed=10)
accept_criterion = lns.accept.RandomAccept(proba=0.5, end_proba=1e-2, step=0.99, method="exponential")

destroy_operators = [
    lns.operators.RandomRemove(
        lns.operators.BasicDestroyConfig(
            dim=problem.dim,
            bounds=[3, 40],
            rng=rng,
        )
    )
]

repair_operators = [
    lns.operators.GreedyRepair(
        lns.operators.BasicRepairConfig(
            problem=problem,
            rng=rng,
        )
    )
]

In [ ]:
seeds = lns.construct.fps_seed(problem.distances, problem.min_vehicles)
seeds

In [ ]:
# initial_solution = lns.construct.nearest_neighbour_builder(problem)
initial_solution = lns.construct.random_builder(problem)

In [ ]:
len(initial_solution.routes)

In [ ]:
opt_sol.cost

In [ ]:
initial_solution.cost

In [ ]:
solver = lns.alns.ALNS(accept=accept_criterion, destroy_operators=destroy_operators, repair_operators=repair_operators)
alns_sol = solver.iterate(initial_solution, max_iter=100_000, max_runtime=30, verbose=True)

In [ ]:
assert cvrp.check_solution(problem, alns_sol.best_solution)

In [ ]:
len(alns_sol.best_solution.routes)

In [ ]:
for x in range(problem.dim):
    matches = [i for i, r in enumerate(initial_solution.routes) if x in r]
    if len(matches) > 1:
        print(f"error: {x} - routes {matches}")

In [ ]:
alns_sol.best_solution.cost

In [ ]:
fig = plot_solution(problem, opt_sol)
py.iplot(fig, filename="opt_solution")

In [ ]:
fig = plot_solution(problem, initial_solution)
py.iplot(fig, filename="initial_nn_solution")

In [ ]:
print(alns_sol.best_solution.cost)
print(len(alns_sol.best_solution.routes))

In [ ]:
fig = plot_solution(problem, alns_sol.best_solution)
py.iplot(fig, filename="alns_best_solution")

In [ ]:
def plot_iterations(solution: lns.alns.TracedSolution, optimal_cost: float):
    df = pd.DataFrame(
        {
            "best": solution.best_costs,
            "running": solution.iteration_costs,
        }
    )

    mape = (solution.best_solution.cost - optimal_cost) / optimal_cost

    fig = px.scatter(
        df,
        y="running",
        height=800,
        width=1200,
        title=f"Solution progress: best MAPE: {mape * 100:.3f}%"
    )

    fig.add_scatter(
        y=df.best,
        mode="lines",
        name="best",
    )

    fig.add_hline(y=optimal_cost)
    fig.update_traces(marker_size=2.5)
    return fig


In [ ]:
fig = plot_iterations(alns_sol, opt_sol.cost)
py.iplot(fig, filename="alns_solution_progress")